In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import requests
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments,EarlyStoppingCallback
import warnings

# Set pandas display options
pd.set_option("display.max_colwidth", 200)

# Ignore warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/Reviews.csv")

In [ ]:
data.drop_duplicates(subset=['Text'],inplace=True)#dropping duplicates
data.dropna(axis=0,inplace=True)#dropping na

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data[['Text', 'Summary']].sample(5)

In [ ]:
stop_words = set(stopwords.words('english'))

def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    # newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [ ]:
#call the function
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0))

In [ ]:
cleaned_text[:5]

In [ ]:
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

In [ ]:
cleaned_summary[:10]

In [ ]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [ ]:
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [ ]:
import matplotlib.pyplot as plt

text_word_count = []
summary_word_count = []

# populate the lists with sentence lengths
for i in data['cleaned_text']:
      text_word_count.append(len(i.split()))

for i in data['cleaned_summary']:
      summary_word_count.append(len(i.split()))

length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})

length_df.hist(bins = 30)
plt.show()

In [ ]:
cnt=0
for i in data['cleaned_summary']:
    if(len(i.split())<=8):
        cnt=cnt+1
print(cnt/len(data['cleaned_summary']))

In [ ]:
max_text_len=30
max_summary_len=8

In [ ]:
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])

df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [ ]:
df.sample(10)

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/preprocessed_df.csv', index=False)

In [ ]:
df.sample(10)

## Loading processed data

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/amazon_food_review_xlnet/preprocessed_df.csv')
df.sample(5)

,text,summary
176962,th birthday gift box big hit party opened music playing contents box exactly expected retro design peace signs outside made box stand need wrap great gift,big hit
213159,influenster quaker kids fight homemade taste great always trustworthy company buying boxes week since first sampled,homemade goodness
189281,fiance starting experiment gluten free mixes products tried first one said definite buy pancakes taste like real pancakes good enough serve guests gluten,great mix definitely buy again
85681,shipping always excellent product amazing family loves white chocolate coconut clusters membership weight watchers preventing ordering frequently,addicting
204016,advertised products met seller advertisement exceeded expectation top notch customer relations team available resolve discrepancies would highly recommend,making case for nuts


In [5]:
df.shape

(214901, 2)

In [6]:
traning_set = df.sample(30000)

In [7]:
train_df, eval_df = train_test_split(traning_set, test_size=0.2, random_state=42)

In [8]:
hf_api = 'c951279b61000ec196651e2204f6d35ad0374e33'

In [9]:
from torch.utils.data import Dataset
import pandas as pd

class TextSummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, text_max_length=30, summary_max_length=8):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.text_max_length = text_max_length
        self.summary_max_length = summary_max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract input text and summary
        text = self.dataframe.iloc[idx]["text"]
        summary = self.dataframe.iloc[idx]["summary"]

        # Tokenize input text
        input_encodings = self.tokenizer(
            text,
            max_length=self.text_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize summary
        target_encodings = self.tokenizer(
            summary,
            max_length=self.summary_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Remove batch dimension
        input_ids = input_encodings["input_ids"].squeeze(0)
        attention_mask = input_encodings["attention_mask"].squeeze(0)
        labels = target_encodings["input_ids"].squeeze(0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

In [10]:
import pandas as pd
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# Create datasets
train_dataset = TextSummarizationDataset(train_df, tokenizer)
eval_dataset = TextSummarizationDataset(eval_df, tokenizer)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [12]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# # Training arguments
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",      # Evaluate at the end of each epoch
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     weight_decay=0.01,
#     save_total_limit=2,
#     num_train_epochs=3,
#     predict_with_generate=True,       # Enable text generation during evaluation
#     logging_dir="./logs"
# )

# # Initialize Trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     tokenizer=tokenizer
# )


# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    remove_unused_columns=False,
    logging_dir='./logs',
    save_total_limit=3,
    fp16=True,
)

# Define early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of epochs with no improvement after which training will stop
    early_stopping_threshold=0.0   # Minimum improvement to consider for early stopping
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Use the evaluation dataset
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

In [ ]:
# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


In [ ]:
sample = train_dataset[0]
print("Input IDs:", sample["input_ids"])
print("Attention Mask:", sample["attention_mask"])
print("Labels:", sample["labels"])